<a href="https://colab.research.google.com/github/forMwish/MyDeepLearn/blob/master/cat_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 准备

## 1.1 挂载 google drive


In [ ]:
from google.colab import drive
import os

gdrive_path = '/gdrive'
drive.mount(gdrive_path, force_remount=True)

os.chdir("%s/MyDrive"%gdrive_path)
os.listdir("%s/MyDrive"%gdrive_path)

## 1.2 配置 kaggle 并下载数据

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp /gdrive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d tongpython/cat-and-dog
!cp /gdrive/MyDrive/cat-and-dog.zip ~/
!mkdir ~/cat-and-dog
!unzip ~/cat-and-dog.zip -d ~/cat-and-dog

## 1.2 数据集处理

In [ ]:
import os, shutil
import numpy as np
import keras
from tensorflow.keras.utils import to_categorical


original_dataset_dir = "/root/cat-and-dog"
base_dir = "/root/cat-and-dog-small"
try:
  shutil.rmtree(base_dir)
except:
  pass
os.mkdir(base_dir, mode=777)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

train_cats_dir = os.path.join(train_dir, "cats")
os.mkdir(train_cats_dir)

train_dogs_dir = os.path.join(train_dir, "dogs")
os.mkdir(train_dogs_dir)

validation_cats_dir = os.path.join(validation_dir, "cats")
os.mkdir(validation_cats_dir)

validation_dogs_dir = os.path.join(validation_dir, "dogs")
os.mkdir(validation_dogs_dir)

test_cats_dir = os.path.join(test_dir, "cats")
os.mkdir(test_cats_dir)

test_dogs_dir = os.path.join(test_dir, "dogs")
os.mkdir(test_dogs_dir)

for file_name in ["cat.%d.jpg"%i for i in range(1, 1001)]:
  src = os.path.join(original_dataset_dir, "training_set/training_set/cats", file_name)
  dst = os.path.join(train_cats_dir, file_name)
  shutil.copyfile(src, dst)

for file_name in ["cat.%d.jpg"%i for i in range(1002, 1502)]:
  src = os.path.join(original_dataset_dir, "training_set/training_set/cats", file_name)
  dst = os.path.join(validation_cats_dir, file_name)
  shutil.copyfile(src, dst)

for file_name in ["cat.%d.jpg"%i for i in range(1503, 2003)]:
  src = os.path.join(original_dataset_dir, "training_set/training_set/cats", file_name)
  dst = os.path.join(test_cats_dir, file_name)
  shutil.copyfile(src, dst)

for file_name in ["dog.%d.jpg"%i for i in range(1, 1001)]:
  src = os.path.join(original_dataset_dir, "training_set/training_set/dogs", file_name)
  dst = os.path.join(train_dogs_dir, file_name)
  shutil.copyfile(src, dst)

for file_name in ["dog.%d.jpg"%i for i in range(1002, 1502)]:
  src = os.path.join(original_dataset_dir, "training_set/training_set/dogs", file_name)
  dst = os.path.join(validation_dogs_dir, file_name)
  shutil.copyfile(src, dst)

for file_name in ["dog.%d.jpg"%i for i in range(1503, 2003)]:
  src = os.path.join(original_dataset_dir, "training_set/training_set/dogs", file_name)
  dst = os.path.join(test_dogs_dir, file_name)
  shutil.copyfile(src, dst)

print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))

print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

# (train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# train_images = train_images.reshape(60000, 28, 28, 1).astype(np.float32) / 255
# test_images = test_images.reshape(10000, 28, 28, 1).astype(np.float32) / 255

# train_labels = to_categorical(train_labels)
# test_labels = to_categorical(test_labels)

# validation_image = train_images[:10000]
# validation_labels = train_labels[:10000]
# partial_train_images = train_images[10000:]
# partial_train_labels = train_labels[10000:]

## 1.4 数据增强测试

In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt

datagen = ImageDataGenerator(
 rotation_range=40,
 width_shift_range=0.2,
 height_shift_range=0.2,
 shear_range=0.5,
 zoom_range=0.2,
 horizontal_flip=True,
 fill_mode='nearest')

fname = [os.path.join(train_cats_dir, fname) for fname in os.listdir(train_cats_dir)]
img_path = fname[0]
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in datagen.flow(x, batch_size=1):
  plt.figure(i)
  imgplot = plt.imshow(image.array_to_img(batch[0]))
  i += 1
  if i % 4 == 0:
    break

plt.show()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.5,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    # classes=['dogs', 'cats'],
    batch_size=20,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,

    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

print(train_generator.class_indices)
print(validation_generator.class_indices)

# 2. 模型

## 2.1 构建

In [ ]:
from keras import models
from keras import layers

from keras import optimizers
from keras import losses
from keras import metrics


network = models.Sequential()
network.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
network.add(layers.MaxPooling2D((2, 2)))
network.add(layers.Conv2D(64, (3, 3), activation='relu'))
network.add(layers.MaxPooling2D((2, 2)))
network.add(layers.Conv2D(128, (3, 3), activation='relu'))
network.add(layers.MaxPooling2D((2, 2)))
network.add(layers.Conv2D(128, (3, 3), activation='relu'))
network.add(layers.MaxPooling2D((2, 2)))
network.add(layers.Flatten())
network.add(layers.Dropout(0.5))
network.add(layers.Dense(512, activation='relu'))
network.add(layers.Dense(1, activation="sigmoid"))
network.compile(optimizer=optimizers.RMSprop(lr=0.001),
        loss=losses.binary_crossentropy,
        metrics=['acc'])
        # metrics=metrics.binary_accuracy)
network.summary()

## 2.2 训练

In [ ]:
network.fit_generator(
      train_generator, 
      steps_per_epoch=100,
      epochs=100, 
      validation_data=validation_generator, 
      validation_steps=50)


# 3 model 数据存储

In [ ]:
import pickle
import os


history_path = './history/cat_dog_v1.0.pickle'
model_path = './model/cat_dog_v1.0'

history_dir = os.path.dirname(history_path)
model_dir = os.path.dirname(model_path)

if not os.path.isdir(history_dir):
  os.mkdir(history_dir)
if not os.path.isdir(model_dir):
  os.mkdir(model_dir)

fp = open(history_path, 'wb')
pickle.dump(network.history.history, fp)
fp.close()

network.save(model_path)

# 4. model 数据恢复

In [ ]:
import pickle

history_path = './history/cat_dog_v1.0.pickle'
fp = open(history_path, 'rb')

history = pickle.load(fp)
fp.close()
model_path = './model/cat_dog_v1.0'
network = models.load_model(model_path)
print(history)

# 5. history 分析


In [ ]:
import matplotlib.pyplot as plt

loss = history['loss']
accuracy = history['acc']
val_loss = history['val_loss']
val_accuracy = history['val_acc']

epoch = range(1, len(loss)+1)

plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.title('loss')
plt.plot(epoch, loss, 'bo', label='loss')
plt.plot(epoch, val_loss, 'r.', label='val_loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.title('accuracy')
plt.plot(epoch, accuracy, 'bo', label='accuracy')
plt.plot(epoch, val_accuracy, 'r.', label='val_accuracy')
plt.show()

## 5.1 测试集验证

In [ ]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print(test_acc)

## 其他

In [ ]:
import matplotlib.pyplot as plt
digit = train_images[4].reshape(28,28)
print(digit.shape)
plt.imshow(digit, cmap=plt.cm.binary)
plt.show()



In [ ]:
for j in range(0, 6, 2):
  print(j)